In [14]:
%pip install pandas
%pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd

possession_file = open("possession_columns.txt", "r")
possession_frames = pd.read_csv("sporting_college_data_1846146_player_possessions.csv")
possession_frames = possession_frames[[col for col in possession_file.read().split("\n")]]

passing_file = open("passing_columns.txt", "r")
pass_frames = pd.read_csv("sporting_college_data_1846146_passing_options.csv")
passing_frames = pass_frames[[col for col in passing_file.read().split("\n")]]


In [16]:
from Player import Player
from Pass import Pass
from TeamPossession import TeamPossession
from PlayerPossession import PlayerPossession
from Zone import Zone

def get_best_option(cur_player, option_index):
    options_frame = passing_frames.iloc[option_index]
    best_frame = None
    best_threat = 0
    pass_exists = False
    while options_frame["player_in_possession_id"] == cur_player:
        cur_xthreat = options_frame["xthreat"] if pd.notnull(options_frame["xthreat"]) else 0
        cur_pass_completion = options_frame["xpass_completion"] if pd.notnull(options_frame["xpass_completion"]) else 0
        if cur_xthreat * cur_pass_completion > best_threat:
            best_threat = cur_xthreat * cur_pass_completion
            best_frame = options_frame
            pass_exists = True
        option_index += 1
        if option_index >= len(passing_frames):
            break
        options_frame = passing_frames.iloc[option_index]
    best_pass = None
    if pass_exists:
        best_pass = Pass(best_frame["pass_direction_id"], best_frame["pass_range_id"], 
                        None, best_frame["xpass_completion"], best_frame["xthreat"], best_frame["player_id"])
    return option_index, best_pass

def get_pass_type(player_frame):
    if player_frame["quick_pass"]:
        return "Quick Pass"
    if player_frame["one_touch"] and not player_frame["is_header"]:
        return "One Touch"
    if player_frame["initiate_give_and_go"]:
        return "Give and Go"
    return "Regular"

def create_player_possession(player_frame, option_index):
    cur_player = Player(player_frame["player_id"], player_frame["player_name"], player_frame["player_position"])
    targeted_pass = Pass(player_frame["interplayer_direction_id"], player_frame["interplayer_distance_range_id"], 
                     get_pass_type(player_frame), player_frame["player_targeted_xpass_completion"], 
                     player_frame["player_targeted_xthreat"], player_frame["player_targeted_name"])
    option_index, best_pass = get_best_option(player_frame["player_id"], option_index)
    start_zone = Zone(player_frame["channel_id_start"], player_frame["third_id_start"])
    end_zone = Zone(player_frame["channel_id_end"], player_frame["third_id_end"])
    return option_index, PlayerPossession(player_frame["separation_start"], player_frame["separation_end"], 
                            start_zone, end_zone, best_pass, 
                            targeted_pass, cur_player, player_frame["frame_start"], player_frame["frame_end"])

possessions = []
# Change team ID
current_team = 2337
option_index = 0
starts_possession = True
cur_possession = None

for i in range(len(possession_frames)):
    cur_frame = possession_frames.iloc[i]
    if cur_frame["team_id"] != current_team:
        if cur_possession:
            possessions.append(cur_possession)
            cur_possession = None
        starts_possession = True
        while option_index < len(passing_frames) and passing_frames.iloc[option_index]["player_in_possession_id"] == cur_frame["player_id"]:
            option_index += 1
        continue
    option_index, player_possession = create_player_possession(cur_frame, option_index)
    if starts_possession:
        cur_possession = TeamPossession()
    cur_possession.add_player_possession(player_possession, starts_possession)
    starts_possession = False

if cur_possession:
    possessions.append(cur_possession)

In [17]:
from MomentDetector import MomentDetector 
from Trends import Trends

md = MomentDetector()
moments_count = 0
up_count = 0
threatening_count = 0
p_moments = []
t_moments = []
for possession in possessions:
   moments = md.detect_moment(possession)
   for moment in moments:
        if moment.moment_type == "Under_Pressure":
            p_moments.append(moment)
        else:
            t_moments.append(moment)
trend = Trends(p_moments, t_moments)
zone_summary = trend.get_zone_insights()
tempo_summary = trend.get_tempo_insights()
switch_summary = trend.get_switching_insights()
player_summary = trend.get_player_chain_insights()


In [18]:
print(zone_summary)
print(tempo_summary)
print(switch_summary)
print(player_summary )

{'zone_chains_failed_pressure': {}, 'starting_zones_failed_pressure': {}, 'zone_chains_successful_pressure': {(10, 10): 7, (8, 8): 6, (5, 10): 3, (9, 8): 3, (3, 1): 3, (3, 8): 3, (6, 6): 3, (6, 7): 3}, 'starting_zones_successful_pressure': {10: 9, 8: 6}, 'zone_chains_threatening': {(15, 13): 2, (12, 13): 2, (11, 13): 2, (13, 13): 2, (9, 13): 2, (13, 12): 2}, 'end_zones_threatening': {13: 4, 12: 3}}
{'tempo_successful_pressure_avg_duration': 10.120000000000001, 'tempo_successful_pressure_avg_one_touch_ratio': 0.41666666666666663, 'tempo_successful_pressure_count': 10, 'tempo_failed_pressure_avg_duration': 0, 'tempo_failed_pressure_avg_one_touch_ratio': 0, 'tempo_failed_pressure_count': 0, 'tempo_threatening_avg_duration': 29.349999999999998, 'tempo_threatening_avg_one_touch_ratio': 0.35, 'tempo_threatening_count': 4}

 Num switches in buildup to attack: 1/4Num switches in escaping pressure: 1/10
{'chains_successful_pressure': {('N. Kuikka', 'J. Joseph'): 3, ('J. Joseph', 'N. Kuikka'): 2

In [19]:
from google import genai

client = genai.Client(api_key="INSERT KEY HERE")

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=
    """
        You are a soccer analyst reviewing data from a team’s performance. The field is divided into 15 zones, composed of 5 vertical channels and 3 horizontal thirds.

        There are two types of situations being analyzed:
        - **Pressure moments**: instances where the team is under high defensive pressure.
        - **Threatening moments**: attacking sequences progressing consecutively toward goal.

        For each situation, the following types of insights are available:
        - **Zones**: which zones possessions start in, end in, and the chains of movement between zones during a possession.
        - **Switches**: switches of play defined as movements across 2 or more channels.
        - **Player chains**: consecutive passes between players.
        - **Player involvement**: how frequently individual players were involved in passes.

        Instructions:
        - Provide a **7-sentence summary** for each situation: one for pressure moments, and one for threatening moments.
        - Present a **bullet-point breakdown** of the key insights in each case, using natural language that’s easy for non-technical readers to understand.
        - Compare related statistics when possible (e.g., overlap between zones used under pressure vs. when attacking, or shared key players).
        - Emphasize patterns in zone transitions, key passing duos, common switching behavior, and tempo if available.

        Only use insights from the data. Write clearly and insightfully, like a coach briefing or match analyst report.
        Here is the data:
        {zone_summary}\n
        {tempo_summary}\n
        {switch_summary}\n
        {player_summary}\n
    """

)

print(response.text)

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}